In [1]:
import carrega_sas

In [2]:
carrega_sas.main()

Lendo dados da base 'DadosComprasNet.sqlite'... OK.
Using SAS Config named: winiomlinux
SAS Connection established. Subprocess id is 18760

Carregando tabela BASE_PREGAO... OK.
Carregando tabela BASE_PREGAO_ITEM... OK.
Carregando tabela BASE_PREGAO_PROPOSTA... Transcoding error encountered.
DataFrame contains characters that can't be transcoded into the SAS session encoding.
'latin-1' codec can't encode character '\u2122' in position 11: ordinal not in range(256)
OK.
Carregando tabela BASE_PREGAO_ADJUDICACAO... OK.
SAS Connection terminated. Subprocess id was 18760

Dados carregados em 0.0h:0.0m:53.04629850387573s.


In [5]:
from config import SQLITE_DB, SAS_LIBREF
import saspy
import sqlite3
import pandas as pd
import time

In [6]:
con = sqlite3.connect(SQLITE_DB)

In [7]:
tabelas = {
    'pregao': 'BASE_PREGAO',
    'item': 'BASE_PREGAO_ITEM',
    'proposta': 'BASE_PREGAO_PROPOSTA',
    'adjudicacao': 'BASE_PREGAO_ADJUDICACAO'
}

In [8]:
def de_sqlite():
    """
    Lê os dados da base sqlite para dataframes.
    :param tabelas: dicionário contendo os nomes das tabelas.
    :return dfs: dicionário contendo os dataframes correspondentes a cada
    tabela.
    """
    dfs = {}
    print(f"Lendo dados da base '{SQLITE_DB}'... ", end='')
    for k, v in tabelas.items():
        dfs[k] = pd.read_sql(f'SELECT * FROM {v}', con, parse_dates=True)
    print('OK.')
    con.close()
    return dfs

In [9]:
dfs = de_sqlite()

Lendo dados da base 'DadosComprasNet.sqlite'... OK.


In [10]:
def prepara_latin1(entrada):
    """
    Substitui caracteres utf-8 que não podem ser codificados em latin_1 por
    similares, para evitar problemas na conversão efetuada pelo SAS.
    :param entrada: string
    :return: string
    """
    saida = entrada
    if isinstance(entrada, str):
        saida = re.sub('[\u201d\u201c]', '"', entrada)
        saida = re.sub('[\u2019\u2018]', '\'', saida)
        saida = re.sub('[\u2013]', '--', saida)
        saida = re.sub('[\u2122]', '\(TM\)', saida)
        saida = re.sub('[\u2022]', '*', saida)
        saida = re.sub('[\u2014]', '--', saida)
        saida = re.sub('[\u02c6]', '^', saida)
        saida = saida.encode('latin1', errors='replace').decode('latin1')
    return saida

In [11]:
import re

In [12]:
dfs['proposta'] = dfs['proposta'].applymap(prepara_latin1)

In [13]:
len(dfs['proposta'])

267985

In [14]:
sas = saspy.SASsession()

Using SAS Config named: winiomlinux
SAS Connection established. Subprocess id is 19824



In [15]:
prop = dfs['proposta']

In [22]:
prop.tail()

,descricaoItem,quantidadeItens,valorEstimadoItem,descricaoComplementarItem,tratamentoDiferenciado,decreto7174,margemPreferencial,unidadeFornecimento,situacaoItem,fornecedorVencedor,...,declaracaoSuperveniente,declaracaoInfantil,declaracaoIndependente,descricaoDeclaracaoCiencia,descricaoMotivoCancelamento,valorClassificado,valorNegociado,observacoes,anexosProposta,idItem
267980,Prestação de Serviço de Vigilância e Segurança...,100,21125.0,SEGURANÇA PARA EVENTOS.,None,Não,Não,Diária,homologado,None,...,Sim,Sim,Sim,None,None,None,None,None,/pregoes/v1/anexo_proposta.html?co_proposta=56...,11010532
267981,Prestação de Serviço de Vigilância e Segurança...,100,21125.0,SEGURANÇA PARA EVENTOS.,None,Não,Não,Diária,homologado,None,...,Sim,Sim,Sim,None,None,None,None,None,/pregoes/v1/anexo_proposta.html?co_proposta=55...,11010532
267982,Prestação de Serviço de Vigilância e Segurança...,100,21125.0,SEGURANÇA PARA EVENTOS.,None,Não,Não,Diária,homologado,None,...,Sim,Sim,Sim,None,None,None,None,None,/pregoes/v1/anexo_proposta.html?co_proposta=56...,11010532
267983,Prestação de Serviço de Vigilância e Segurança...,100,21125.0,SEGURANÇA PARA EVENTOS.,None,Não,Não,Diária,homologado,None,...,Sim,Sim,Sim,None,None,None,None,None,/pregoes/v1/anexo_proposta.html?co_proposta=56...,11010532
267984,Prestação de Serviço de Vigilância e Segurança...,100,21125.0,SEGURANÇA PARA EVENTOS.,None,Não,Não,Diária,homologado,None,...,Sim,Sim,Sim,None,None,None,None,None,/pregoes/v1/anexo_proposta.html?co_proposta=56...,11010532


In [23]:
sas.df2sd(df=prop, table='BASE_PREGAO_PROPOSTA', libref='LIB_MATR')

Libref  = LIB_MATR
Table   = BASE_PREGAO_PROPOSTA
Dsopts  = {}
Results = Pandas

In [24]:
prop2 = sas.sd2df(table='BASE_PREGAO_PROPOSTA', libref='LIB_MATR')

In [25]:
len(prop2)

1559

In [27]:
log = sas.saslog()

In [30]:
log[log.find('ERROR'):log.find('ERROR')+500]

'ERROR: INPUT statement exceeded record length. INFILE CARDS OPTION STOPOVER specified.\nRULE:      ----+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8----+----9----+----0                     \n\n2198CHAR   ALMOFADA CARIMBO.250.1883.0.ALMOFADA CARIMBO, MATERIAL CAIXA PLÁSTICO/METAL, TAMANHO Nº 3, COR AZUL,\n    ZONE   444444442445444403330333323044444444244544442244545444244454254C5544424454422544444424B2322445245542\n    NUMR   1CDF614103129D2F325031883E031CDF614103129D2F'

In [33]:
log

'\x0c1                                                          The SAS System                              18:24 Monday, August 26, 2019\n\nNOTE: Copyright (c) 2002-2012 by SAS Institute Inc., Cary, NC, USA. \nNOTE: SAS (r) Proprietary Software 9.4 (TS1M4) \n      Licensed to TC DF - TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, Site 70201676.\nNOTE: This session is executing on the Linux 2.6.32-642.el6.x86_64 (LIN X64) platform.\n\n\n\nNOTE: Updated analytical products:\n      \n      SAS/STAT 14.2\n      SAS/ETS 14.2\n\nNOTE: Additional host information:\n\n Linux LIN X64 2.6.32-642.el6.x86_64 #1 SMP Tue May 10 17:27:01 UTC 2016 x86_64 CentOS release 6.8 (Final) \n\nYou are running SAS 9. Some SAS 8 files will be automatically converted \nby the V9 engine; others are incompatible.  Please see \nhttp://support.sas.com/rnd/migration/planning/platform/64bit.html\n\nPROC MIGRATE will preserve current SAS file attributes and is \nrecommended for converting all your SAS libraries from any \nSAS

In [32]:
prop.loc[prop['descricaoComplementarItem'].str.contains('ALMOFADA CARIMBO, MATERIAL CAIXA PLÁSTICO/METAL, TAMANHO Nº 3, COR AZUL')]

ValueError: cannot index with vector containing NA / NaN values

In [20]:
len(prop2)

1559

In [32]:
sas.endsas()

SAS Connection terminated. Subprocess id was 21304


In [37]:
'\u02c6Flávio'.encode('latin1', errors='replace').decode('latin1')

'?Flávio'

In [34]:
help(str.encode)

Help on method_descriptor:

encode(self, /, encoding='utf-8', errors='strict')
    Encode the string using the codec registered for encoding.
    
    encoding
      The encoding in which to encode the string.
    errors
      The error handling scheme to use for encoding errors.
      The default is 'strict' meaning that encoding errors raise a
      UnicodeEncodeError.  Other possible values are 'ignore', 'replace' and
      'xmlcharrefreplace' as well as any other name registered with
      codecs.register_error that can handle UnicodeEncodeErrors.



In [23]:
def para_sas(dfs):
    """
    Carrega os dataframes pandas para a library do SAS.
    :param dfs: dicionário contendo os dataframes a serem carregados
    :return: none
    """
    sas = saspy.SASsession()
    for k, v in dfs.items():
        print(f'Carregando tabela {tabelas[k]}... ', end='')
        sas.df2sd(df=dfs[k], table=tabelas[k], libref=SAS_LIBREF)
        print('OK.')
    sas.disconnect()
    sas.endsas()
    return

In [27]:
sas = saspy.SASsession()

Using SAS Config named: winiomlinux
SAS Connection established. Subprocess id is 19144



In [28]:
sas.assigned_librefs()

['WORK',
 'SASDATA',
 'LIB_MATR',
 'RFB',
 'LIB_NIE',
 'ECONTAS',
 'SEGECEX',
 'PESSOAS',
 'SANCAO',
 'RAIS',
 'SISEDITA',
 'STPSAMP',
 'FCMENRCH',
 'FCM_SLIB',
 'FCMPREP',
 'FCMATEMP',
 'SASHELP',
 'MAPS',
 'MAPSSAS',
 'MAPSGFK',
 'SASUSER']

In [29]:
SAS_LIBREF

'LIBMATR'